In [2]:
from lightgbmlss.model import *
from lightgbmlss.distributions.Gaussian import *
from lightgbmlss.distributions.Mixture import *
from lightgbmlss.distributions.mixture_distribution_utils import MixtureDistributionClass

from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
figure_size = (10,5)
import plotnine
from plotnine import *
import pandas as pd
import numpy as np
import sqlite3
plotnine.options.figure_size = figure_size

conn = sqlite3.connect('../data/nfp2/nfp2.db')  # Adjust the path to your database file

# Step 2 & 3: Query the database and load the data into a pandas DataFrame
query = "SELECT * FROM stellarators"  # Adjust your query as needed
data_df = pd.read_sql_query(query, conn)

data_df_clean = data_df[data_df['quasisymmetry'] <= 10]
data_df_clean = data_df_clean.dropna(subset=['quasisymmetry'])


X = data_df_clean[['rbc_1_0', 'rbc_m1_1', 'rbc_0_1', 'rbc_1_1','zbs_1_0', 'zbs_m1_1', 'zbs_0_1', 'zbs_1_1']] 
Y = np.log(data_df_clean['quasisymmetry'])

features_no_outliers, test_features_no_outliers, target_no_outliers, test_target_no_outliers = train_test_split(X, Y, test_size=0.2, random_state=42)


: 

In [ ]:
dtrain = lgb.Dataset(features_no_outliers, label=target_no_outliers.values)

# Specifies a mixture of Gaussians. See ?Mixture for an overview.
lgblss = LightGBMLSS(
    Mixture(
        Gaussian(response_fn="softplus", stabilization="L2"), 
        M = 4,
        tau=1.0,
        hessian_mode="individual",
    )
)

In [ ]:
param_dict = {
    "max_depth":                ["int",   {"low": 1,      "high": 10,    "log": False}],
    "min_gain_to_split":        ["float", {"low": 1e-8,   "high": 40,    "log": True}],
    "min_sum_hessian_in_leaf":  ["float", {"low": 1e-8,   "high": 500,   "log": True}],
    "subsample":                ["float", {"low": 0.2,    "high": 1.0,   "log": False}],
    "feature_fraction":         ["float", {"low": 0.2,    "high": 1.0,   "log": False}],
    "boosting":                 ["categorical", ["gbdt"]],
}

np.random.seed(123)
opt_param = lgblss.hyper_opt(param_dict,
                             dtrain,
                             num_boost_round=100,        # Number of boosting iterations.
                             nfold=5,                    # Number of cv-folds.
                             early_stopping_rounds=20,   # Number of early-stopping rounds
                             max_minutes=60,             # Time budget in minutes, i.e., stop study after the given number of minutes.
                             n_trials=20,                # The number of trials. If this argument is set to None, there is no limitation on the number of trials.
                             silence=True,               # Controls the verbosity of the trail, i.e., user can silence the outputs of the trail.
                             seed=123,                   # Seed used to generate cv-folds.
                             hp_seed=None                # Seed for random number generator used in the Bayesian hyperparameter search.
                            )

  0%|          | 0/20 [00:00<?, ?it/s]


[W 2024-05-29 23:33:51,621] Trial 0 failed with parameters: {'eta': 0.02321577394982007, 'max_depth': 1, 'min_gain_to_split': 1.9175695483926406e-05, 'min_sum_hessian_in_leaf': 0.004033644983726181, 'subsample': 0.32411478494421386, 'feature_fraction': 0.919693549940257, 'boosting': 'gbdt'} because of the following error: ValueError("could not determine the shape of object type 'Series'").
Traceback (most recent call last):
  File "/home/rofarate/PIC-STELLARATOR/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/home/rofarate/PIC-STELLARATOR/.venv/lib/python3.10/site-packages/lightgbmlss/model.py", line 387, in objective
    lgblss_param_tuning = self.cv(hyper_params,
  File "/home/rofarate/PIC-STELLARATOR/.venv/lib/python3.10/site-packages/lightgbmlss/model.py", line 267, in cv
    self.set_init_score(train_set)
  File "/home/rofarate/PIC-STELLARATOR/.venv/lib/python3.10/site-packages/lightgbmlss/model.py",

ValueError: could not determine the shape of object type 'Series'

In [ ]:
np.random.seed(123)

opt_params = opt_param.copy()
n_rounds = opt_params["opt_rounds"]
del opt_params["opt_rounds"]

# Train Model with optimized hyperparameters
lgblss.train(opt_params,
             dtrain,
             num_boost_round=n_rounds,
             )